In [ ]:
# this notebook is used to explore the data we have around diagnosis

In [ ]:
# cross check dates of first azotemia, low USG with diagnoses
# come up with scheme for first diagnosed date
# generally look at what diagnosis info we have
# add a column to diagnose based on IRIS reference ranges
# add a column for Rachel's diagnosis reference range for species she calculated

In [ ]:
# 'Date of Onset of Azoatemia'  completely empty
# 'Date of USG less than 1,035' empty
# 'Onset of azotaemia' these are dates and there are 56 of them

In [ ]:
# SCHEME FOR INFERRING DATE OF DIAGNOSIS
# logic_flag
# 0: can't infer
# 1: date of diagnosis (4 cats)
# 2. 'First report of CKD CS' (7)
# 3. Ante-mortem Dx TRUE and use earliest 'Onset of azotaemia' / 'Date of first low USG'
# 4. Died of CKD  and use earliest 'Onset of azotaemia' / 'Date of first low USG'

# sometimes the first indicator date is the same as date of death.  Once labeling is done will need to compare date of death and fisrt indicator date and consider excluding record if these are too close in time.


In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# read in unified animal list
file = '../Data/big_cats/Access_DB_table_exports/PrevalenceIncludedFelidsTZ_ZV_WA_SA_MG_DM.xlsx'
data = pd.read_excel(file)

In [3]:
data.columns

Index(['Animal Name', 'ARKS Number', 'Microchip Number',
       'Species (common name)', 'Sex', 'Alternative ARKS Nos',
       'Studbook Number', 'GAN ID', 'Status (alive/deceased/transacted)',
       'Zoo Location', 'Date  of Birth', 'Date of Transaction',
       'Date of Death', 'Age in years at death', 'Date of Onset of Azoatemia',
       'Date of USG less than 1,035', 'Ante-mortem Dx of CKD',
       'Date of first low USG', 'Onset of azotaemia', 'Date of diagnosis',
       'Comments re kidney function', 'First report of CKD CS', 'Age at Death',
       'Cause of Death', 'Acute kidney failure', 'CKD - cause of mortality',
       'CKD - co-morbidity', 'Pyometra (survived)', 'Pyometra cause of death',
       'Unknown', 'Neonatal', 'Neurological', 'Endocrine', 'Neoplasia',
       'Old Age debility', 'Trauma', 'Infection', 'GIT',
       'Respiratory/Cardiovascular', 'Pathology Report Numbers',
       'Post Mortem Report Number', 'International transaction',
       'Dam ARKS Number', 'Sir

In [10]:
diagnosis_date_labeling = data[['ARKS Number', 'Ante-mortem Dx of CKD', 'Date of first low USG', 'Onset of azotaemia', 'Date of diagnosis', 'First report of CKD CS', 'Cause of Death', 'CKD - cause of mortality', 'CKD - cause of mortality', 'Date of Death']]
diagnosis_date_labeling

,ARKS Number,Ante-mortem Dx of CKD,Date of first low USG,Onset of azotaemia,Date of diagnosis,First report of CKD CS,Cause of Death,CKD - cause of mortality,CKD - cause of mortality,Date of Death
0,800100,False,NaT,NaT,NaT,NaN,euthanasia- renal failure,True,True,1982-04-24
1,A49221,False,2013-04-04,2013-03-03,NaT,NaN,CKD and corneal ulcer,True,True,2013-04-04
2,890000,True,1998-05-28,2001-03-01,2001-03-01,NaN,CKD,True,True,2001-07-03
3,K46,True,NaT,NaT,1973-01-23,NaN,CKD,True,True,1973-05-23
4,K0095,False,NaT,NaT,NaT,NaN,"kidney stones, haematuria, old age",True,True,1986-12-24
5,900248,False,NaT,NaT,NaT,NaN,CKD,True,True,2004-10-28
6,730024,True,NaT,NaT,NaT,NaN,"CKD, pulmonary adenocarcinoma",True,True,1990-05-01
7,A20021,True,2012-09-21,2015-08-24,NaT,NaN,pyelonephritis,True,True,2015-08-24
8,921430,False,2000-03-28,2001-05-29,NaT,NaN,"TIN R kidney and chronic cystitis, nephrolith,...",True,True,2001-06-18
9,820016,False,NaT,NaT,NaT,NaN,glomerulonephritis and glomerulosclerosis,True,True,1989-06-27


In [12]:
# write to csv to label
file = '../Data/big_cats/processed_data/diagnosis_date_labeling.csv'
diagnosis_info.to_csv(file, index=False)

In [6]:
len(data)

424

In [7]:
len(data['ARKS Number'].unique())

424

In [8]:
data['Ante-mortem Dx of CKD'].value_counts()

False    410
True      14
Name: Ante-mortem Dx of CKD, dtype: int64

In [9]:
data['CKD - cause of mortality'].value_counts()

False    348
True      76
Name: CKD - cause of mortality, dtype: int64

In [14]:
len(data[(data['CKD - cause of mortality'] == True) & (data['Ante-mortem Dx of CKD'] == True)])

11

In [15]:
data['Date of diagnosis'].value_counts()

2011-11-16    1
2013-03-07    1
2001-03-01    1
1973-01-23    1
Name: Date of diagnosis, dtype: int64

In [17]:
data['First report of CKD CS'].value_counts()

28-jul-10    2
20-3-92      1
25-10-02     1
21-Oct-94    1
22-may-08    1
16-may-99    1
Name: First report of CKD CS, dtype: int64

In [7]:
data['Onset of azotaemia'].value_counts().sum()

56

In [21]:
data['Date of first low USG'].value_counts().sum()

39

In [22]:
data['Date of USG less than 1,035'].value_counts()

Series([], Name: Date of USG less than 1,035, dtype: int64)

In [5]:
data['Date of Onset of Azoatemia'].value_counts()

Series([], Name: Date of Onset of Azoatemia, dtype: int64)